In [1]:
import pandas as pd                        # Importe la bibliothèque pandas et la renomme en pd
import spacy                               # Importe la bibliothèque spacy
import numpy as np                         # Importe la bibliothèque numpy et la renomme en np
import surprise
import os
from surprise import Dataset, Reader, KNNWithZScore  # Importe les classes Dataset, Reader et KNNWithZScore depuis la bibliothèque surprise
from collections import defaultdict       # Importe la classe defaultdict depuis la bibliothèque collections
import openai

# Préparation des données pour surprise
from surprise import Reader, Dataset      # Importe les classes Reader et Dataset depuis la bibliothèque surprise
from surprise.prediction_algorithms.knns import KNNWithZScore  # Importe la classe KNNWithZScore depuis le module knns de la bibliothèque surprise


In [2]:
data = pd.read_csv("BDD/avis_sans_outliers.csv")

In [3]:
# Drop unnecessary columns and handle missing values
data_cleaned = data.drop(columns=['Unnamed: 0', 'url', 'title_review', 'date_published'])
data_cleaned['comment'] = data_cleaned['comment'].fillna('')  # Fill missing comments
print(np.__version__)
print(surprise.__version__)

1.26.4
1.1.3


In [4]:
from surprise import Dataset, Reader, KNNWithZScore
from collections import defaultdict

# Prepare data for surprise
reader = Reader(rating_scale=(1, 10))  # Assuming rating scale is from 1 to 10
data_surprise = Dataset.load_from_df(data_cleaned[['author', 'title', 'note']], reader)

# Build full trainset
trainset = data_surprise.build_full_trainset()

# Initialize KNN with Z-Score algorithm for user-based collaborative filtering
algo = KNNWithZScore(sim_options={'name': 'cosine', 'user_based': True}, k=20, min_k=1)
algo.fit(trainset)

def get_neighbors(user_id, game_title, k=20):
    # Retrieve inner ID of the user
    user_inner_id = trainset.to_inner_uid(user_id)
    
    # Retrieve inner ID of the game
    try:
        game_inner_id = trainset.to_inner_iid(game_title)
    except ValueError:
        return f"No data available for the game '{game_title}'."
    
    # Retrieve the k nearest neighbors of the user
    neighbors = algo.get_neighbors(user_inner_id, k)
    # Convert inner IDs of the neighbors back to raw IDs
    neighbors_ids = [trainset.to_raw_uid(inner_id) for inner_id in neighbors]
    print(neighbors_ids)
    # Filter the dataset to find the neighbors who have rated the specified game
    filtered_comments = data_cleaned[(data_cleaned['author'].isin(neighbors_ids)) & (data_cleaned['title'] == game_title)]
    
    # Collect and return usernames and their comments on the specified game
    return filtered_comments[['author', 'comment']].values.tolist()


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [5]:
neighbors_comments = get_neighbors('Monsieur Guillaume', 'Mariposas', k=20)
with open('output_comments.txt', 'w') as file:
    for c in neighbors_comments :
        file.write(str(c)+"\n")
        

['prunelles', 'Olfenw', 'Lilly', 'letroyenfou', 'Empédocle', 'Ricardo31', 'Player One', 'Sempre Sainté', 'grotesk', 'Olène', 'Greta', 'Patmol', 'Delorean', 'plumesdanges', 'Didi5962', 'Sirien', 'Pollo', 'Meeplejuice', 'Gido_L4', 'Tigragon22']


In [6]:
cs = get_neighbors('Monsieur Guillaume','Mariposas')
for c in cs:
    print(c)
    print()

['prunelles', 'Olfenw', 'Lilly', 'letroyenfou', 'Empédocle', 'Ricardo31', 'Player One', 'Sempre Sainté', 'grotesk', 'Olène', 'Greta', 'Patmol', 'Delorean', 'plumesdanges', 'Didi5962', 'Sirien', 'Pollo', 'Meeplejuice', 'Gido_L4', 'Tigragon22']
['prunelles', 'Très joli bijou que ce jeu-là ! Le matériel, les illustrations, l\'histoire et le propos du jeu, tout donne envie de contribuer à sauver ces papillons.\\nBelle courbe d\'apprentissage, impossible de tout faire même si on essaie parfois.\\nLe mini bemol serait les papillons qu\'on n\'arrive pas toujours à visualiser parce que certaines couleurs sont proches mais en même ça nous permet de les camoufler aussi aux yeux de nos adversaires.\\nPensez à aller regarder les différentes variantes concernant les cartes "cycle de vie". \\nUne belle réussite !']

['Olfenw', '']

['Lilly', "Mariposas est un jeu ou la tension va s'exacerber au cours de la partie, ou devrais-je dire : au fil des saisons.\\n\\nOn joue des papillons, on se balade, on 

In [7]:
def filter_comments_by_word_count(comments, min_word_count):
    """
    Filters a list of comments to include only those with a word count greater or equal to min_word_count.
    
    Args:
    comments (list of str): A list of comments.
    min_word_count (int): The minimum number of words required for a comment to be included in the return list.
    
    Returns:
    list of str: A list containing only comments that meet or exceed the word count requirement.
    """
    # Filter comments based on word count
    filtered_comments = [comment for comment in comments if len(comment.split()) >= min_word_count]
    return filtered_comments

In [8]:
def get_top_comments(user_id, game_title, N, word_count_threshold):
    """
    Retrieves the top N comments from the nearest neighbors of a specified user about a specific game,
    using the KNNWithZScore algorithm from Surprise. Only comments with a word count above a certain threshold are considered.
    Expands the search if initial neighbors have not commented on the game. Includes the rank of each commenting user based on proximity.

    Args:
    user_id (str): The user ID of the interested user.
    game_title (str): The title of the game for which comments are being retrieved.
    N (int): The number of top comments to return based on relevance.
    word_count_threshold (int): The minimum number of words required for comments to be considered.

    Returns:
    list of tuples: A list containing tuples of (rank, author, comment) if available, or a status message.
    """
    try:
        # Retrieve inner ID of the user
        user_inner_id = trainset.to_inner_uid(user_id)
    except ValueError:
        return f"No data available for the user ID '{user_id}'."

    try:
        # Retrieve inner ID of the game
        game_inner_id = trainset.to_inner_iid(game_title)
    except ValueError:
        return f"No data available for the game '{game_title}'."

    # Initialize variables for searching neighbors and tracking unique comments
    k = 20
    max_neighbors = trainset.n_users  # Maximum possible neighbors
    found_comments = []
    processed_neighbor_ids = set()  # Set to track processed neighbors

    # Retrieve neighbors and expand search until enough comments are found or all users are checked
    while len(found_comments) < N and k <= max_neighbors:
        # Retrieve the k nearest neighbors of the user
        neighbors = algo.get_neighbors(user_inner_id, k=k)
        # Convert inner IDs of the neighbors back to raw IDs and store with ranks
        neighbors_with_ranks = [(rank + 1, trainset.to_raw_uid(inner_id)) for rank, inner_id in enumerate(neighbors) if trainset.to_raw_uid(inner_id) not in processed_neighbor_ids]

        # Filter the dataset to find the neighbors who have rated the specified game
        for rank, neighbor_id in neighbors_with_ranks:
            if neighbor_id not in processed_neighbor_ids:
                processed_neighbor_ids.add(neighbor_id)
                neighbor_comments = data_cleaned[(data_cleaned['author'] == neighbor_id) & (data_cleaned['title'] == game_title)]
                # Apply enhanced word count filter
                valid_comments = filter_comments_by_word_count(neighbor_comments['comment'].tolist(), word_count_threshold)
                for comment in valid_comments:
                    found_comments.append((rank, neighbor_id, comment))

        # Increase the number of neighbors for the next iteration if necessary
        k += 20

    # If comments are found, sort them by length and return the top N
    if found_comments:
        found_comments.sort(key=lambda x: len(x[2]), reverse=True)  # Sort by comment length
        return found_comments[:N]
    else:
        return f"No comments found for the game '{game_title}' from nearest neighbors that meet the word threshold."


In [9]:
data_cleaned.head()

,author,note,title,comment
0,Monsieur Guillaume,8,Mariposas,"Lorsque le jeu est jeu, bon, réflexif, joli po..."
1,morlockbob,7,Mariposas,Comment continuer après un mega hit ? Simpleme...
2,SwatSh,7,Mariposas,"Vin d'jeu: Avec Mariposas, Elizabeth Hargrave ..."
3,Timi JeuxATheme,8,Mariposas,
4,prunelles,9,Mariposas,"Très joli bijou que ce jeu-là ! Le matériel, l..."


In [10]:
top_comments_with_rank = get_top_comments('Monsieur Guillaume', 'Invasions', 5, 10)
for comment in top_comments_with_rank:
    print(comment)
    print()

(295, 'ioucounou', 'Voilà exactement le jeu que j’adore. L’un des meilleurs sortis cette année, selon moi.Invasions est typiquement le genre de jeu qui a été développé autour de son thème. Et ça se sent ! Tout est fait pour qu’on s’y croit. C’est un jeu épique où l’on planifie ses campagnes avec soins. Vais-je aller piller la côte anglaise ? Ou bien faire du commerce dans l’est ? Les possibilités sont nombreuses et toutes requiert une préparation minutieuse sous peine de perdre un temps précieux en aller/retour ! Il faut anticiper la météo, prévoir combien de guerriers seront nécessaires, et surtout, et c’est là que le jeu mérite son 5/5, il faut toujours essayer d’anticiper ce que vont faire les autres jarls !En effet, Invasions est aussi un jeu interactifs, où l’on peut être très méchant.Toute la finesse du jeu consiste à profiter de la moindre faiblesse dans ce que font les autres. Et cela engendre à tous les coups rancœurs et vengeances mémorables.Bref, Invasions est un jeu à thème

In [11]:
data_cleaned['author'].nunique()

2458

In [12]:
MG=data_cleaned[['title','comment','note']].loc[data_cleaned['author']=='Monsieur Guillaume']
list=MG['comment'].loc[MG['note']==10].tolist()
for c in list:
    print(c)

... mais jamais ne faiblira car toujours là je serai pour ce jeu aux potentiels infinis (ou presque) et tous aussi enthousiasmant les uns que les autres...Du très bon matériel (avec la technique de l'eau chaude, tout devient encore plus beau et plus droit), des règles qui en donnent beaucoups et qui en promettent encore plus, des créatures, des conseils de guerres, des tuiles qu'on rêvent de voir, d'autres qu'on a qu'on rêve de mettre en 3D...Non, ce jeu, c'est du rêve, de l'amusement puissance dix en boîte... et j'attends mars sans impatience pour les prochaines extensions car jusque là, j'ai déjà largement de quoi faire dans une boite qui peut se suffire à elle-même pour les plus récalcitrant...que du bonheur!!
En effet, c'est un bonheur pour moi d'avoir de la place, de manoeuvrer avec des armées plus grandes, de pouvoir jouer plus longtemps...Bon, je pourrais jouer sans, bien sûr, car justement, maintenant, le jeu est modulable à souhait en fonction du temps, du niveau, etc... voulu

In [13]:
MG[['title','comment']].loc[MG['note']==10]

,title,comment
32239,BattleLore Seconde Édition,... mais jamais ne faiblira car toujours là je...
86158,Epic BattleLore : Batailles épiques,"En effet, c'est un bonheur pour moi d'avoir de..."
87387,Invasions,"Au départ, les retours effectués en coulant so..."
88107,Marvel Heroes,"... mais collé à la boite...En effet, le côté ..."
91155,Niagara : Flussgeister am Niagara,"et bien et bien... moi qui était, avec toute l..."
98297,Niagara,"... en fait, il y a tout pour passer un bon mo..."
116893,Mr. Jack - London,... et je pèse mes mots!!Je trouve ce jeu très...


In [14]:
# Charger le modèle de langue française
nlp = spacy.load('fr_core_news_sm')

# Fonction adaptée pour traiter chaque commentaire individuellement
def filtrer_commentaire(commentaire):
    doc = nlp(commentaire)
    pos_exclues = ['DET', 'CONJ', 'PRON', 'ADP', 'CCONJ', 'PUNCT']  # Ajout de 'PUNCT' à la liste des exclusions
    #mots_exclus = ['jeu', 'jeux'] and token.lemma_.lower() not in mots_exclus
    # Filtrer les tokens qui ne sont pas dans pos_exclues et dont le lemme n'est pas dans mots_exclus
    mots_filtres = [token.text for token in doc if token.pos_ not in pos_exclues ]
    return ' '.join(mots_filtres)
# S'assurer que tous les commentaires sont des chaînes de caractères
MG['comment'] = MG['comment'].astype(str)

In [15]:
MG['comment_filtrer'] = MG['comment'].apply(filtrer_commentaire)
print(MG[['comment_filtrer']].head())

                                       comment_filtrer
0    Lorsque jeu est jeu bon réflexif joli est sens...
73   Edition Univers Direction artistique Matériel ...
205  Edition Univers Direction artistique Matériel ...
216  Marions est cas dire thème finalement peu usit...
308  Oui parce que là faudra peu masser neurones au...


In [16]:
def tokeniser(commentaire):
    doc = nlp(commentaire)
    tokens = [token.text for token in doc]
    return tokens

# Appliquer la tokenisation sur la colonne des commentaires traités
MG['tokens'] = MG['comment_filtrer'].apply(tokeniser)


In [17]:
print(MG['tokens'])

0         [Lorsque, jeu, est, jeu, bon, réflexif, joli, ...
73        [Edition, Univers, Direction, artistique, Maté...
205       [Edition, Univers, Direction, artistique, Maté...
216       [Marions, est, cas, dire, thème, finalement, p...
308       [Oui, parce, que, là, faudra, peu, masser, neu...
1046      [est, Renegade, France, repartons, recherche, ...
1426      [Edition, Univers, Direction, artistique, Maté...
1587      [Edition, Univers, Direction, artistique, Maté...
1649      [Très, bien, vu, Monsieur, Sing, console, Cap,...
1825      [Edition, Univers, Direction, artistique, Maté...
1937      [Edition, Univers, Direction, artistique, Maté...
2345      [Largement, faire, autour, jeu, permet, retrou...
2424      [Petit, jeu, mot, extension, Heroquest, aura, ...
2435      [plaisir, mécanisme, peu, exploité, aimants, m...
2703      [Edition, Univers, Direction, artistique, Maté...
2760      [jeu, agréable, dynamique, attention, léger, s...
2807      [Ah, plaisir, humain, ranger, 

In [24]:
total_tokens = sum(len(tokens) for tokens in MG['tokens'])
print("Nombre total de tokens:", total_tokens)


Nombre total de tokens: 5106


In [22]:
def generate_aggregated_comments(dataframe, api_key, model="gpt-3.5-turbo"):
    # Concaténation de tous les tokens en une seule liste
    all_tokens = []
    for tokens in dataframe['tokens']:
        all_tokens.extend(tokens)
    
    # Conversion de la liste de tokens en une chaîne de caractères
    token_string = " ".join(all_tokens)
    
    # Initialisation du client OpenAI
    client = openai.OpenAI(api_key=api_key)
    
    # Générer un commentaire positif
    positive_response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": f"Generate a positive comment based on: {token_string}"}]
    )
    
    # Générer un commentaire négatif
    negative_response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": f"Generate a negative comment based on: {token_string}"}]
    )
    
    return positive_response.choices[0].message['content'], negative_response.choices[0].message['content']

# Exemple d'utilisation
api_key = "sk-proj-17CQuXWNpWoNU2vCjxXUT3BlbkFJ6KFmtiG506MzO1QCcbBy"
positive_comment, negative_comment = generate_aggregated_comments(MG, api_key)
print("Commentaire Positif:", positive_comment)
print("Commentaire Négatif:", negative_comment)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [18]:
def generate_comments_for_each_list(dataframe, model="gpt-3.5-turbo", api_key="Votre_Clé_API"):
    """
    Génère un commentaire positif et un commentaire négatif pour chaque liste de tokens dans un DataFrame.
    
    Args:
    dataframe (DataFrame): DataFrame contenant les listes de tokens.
    model (str): Modèle GPT à utiliser.
    api_key (str): Clé API pour OpenAI.
    
    Returns:
    DataFrame: DataFrame contenant les commentaires positifs et négatifs générés pour chaque liste.
    """
    # Initialisation du client OpenAI avec la clé API
    client = openai.OpenAI(api_key=api_key)
    comments = []

    for token_list in dataframe['tokens']:
        if token_list:  # Assure que la liste de tokens n'est pas vide
            context = " ".join(token_list)

            # Générer un commentaire positif
            positive_response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": f"Commentaire positif basé sur les mots-clés suivants: {context}"}]
            )

            # Générer un commentaire négatif
            negative_response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": f"Commentaire négatif basé sur les mots-clés suivants: {context}"}]
            )

            # Ajouter les résultats au tableau des commentaires
            comments.append({
                "tokens": context,
                "positive_comment": positive_response['choices'][0]['message']['content'].strip(),
                "negative_comment": negative_response['choices'][0]['message']['content'].strip()
            })

    return pd.DataFrame(comments)



In [19]:
print(openai.__version__)

1.25.0


In [20]:
# Remplacez 'Votre_Cle_API' par votre véritable clé API
api_key = "sk-proj-17CQuXWNpWoNU2vCjxXUT3BlbkFJ6KFmtiG506MzO1QCcbBy"

# Appel de la fonction pour générer les commentaires
results = generate_comments_for_each_list(MG, model="gpt-3.5-turbo", api_key=api_key)

# Afficher les résultats
print(results)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}